# Set up `Observation` objects for each Exfor subentry

Internally, this involves initializing a `jitr.xs.elastic.DifferentialWorkspace` for each `Observation`, which precomputes things like boundary conditions to speed up computation of observables for an interaction parameter.

In [4]:
import numpy as np

In [5]:
import rxmc

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [6]:
import lzma
import pickle
from pathlib import Path

from tqdm import tqdm

output_dir = Path("./data/")

In [7]:
with lzma.open(output_dir / "pp_elastic_data.xz", "rb") as f:
    pp_elastic_measurements = pickle.load(f)
with lzma.open(output_dir / "nn_elastic_data.xz", "rb") as f:
    nn_elastic_measurements = pickle.load(f)

- statistical errors only
- covariance fixed so use `FixedCovarianceObservation` to ensure cov matrix is precomputed

In [10]:
nn_observations = {}
for entry_id, mlist in tqdm(nn_elastic_measurements.items()):
    if entry_id not in nn_observations:
        nn_observations[entry_id] = []
    for rxn, m in mlist:
        nn_observations[entry_id].append(
            rxmc.elastic_diffxs_observation.ElasticDifferentialXSObservation(
                measurement=m,
                reaction=rxn,
                quantity="dXS/dA",
                lmax=50,
                angles_vis=np.linspace(max(0,np.min(m.x)*0.9), min(180,np.max(m.x)*1.1), 100),
                ObservationClass=rxmc.observation.FixedCovarianceObservation,
                error_kwargs={
                    "include_sys_norm_err": False,
                    "include_sys_offset_err": False,
                },
            )
        )

100%|███████████████████████████████████████████████████████████████| 18/18 [02:21<00:00,  7.87s/it]


In [11]:
pp_observations = {}
for  entry_id, mlist in tqdm(pp_elastic_measurements.items()):
    if entry_id not in pp_observations:
        pp_observations[entry_id] = []
    for rxn, m in mlist:
        pp_observations[entry_id].append(
            rxmc.elastic_diffxs_observation.ElasticDifferentialXSObservation(
                m,
                rxn,
                "dXS/dRuth",
                50,                
                angles_vis=np.linspace(max(0,np.min(m.x)*0.9), min(180,np.max(m.x)*1.1), 100),
                ObservationClass=rxmc.observation.FixedCovarianceObservation,
                error_kwargs={
                    "include_sys_norm_err": False,
                    "include_sys_offset_err": False,
                },
            )
        )

100%|███████████████████████████████████████████████████████████████| 55/55 [05:39<00:00,  6.17s/it]


In [12]:
with lzma.open(output_dir / "pp_elastic_observations.xz", "wb") as f:
    pickle.dump(pp_observations, f)
with lzma.open(output_dir / "nn_elastic_observations.xz", "wb") as f:
    pickle.dump(nn_observations, f)